In [7]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNNV2, RNN, RNNV12
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager, select_label
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
log_dir = './train_log/separate/test-1l/'
logger.set_logger_dir(log_dir, action='d')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[1104 09:11:44 @logger.py:74] Argv: /home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1007/jupyter/kernel-f2c54482-45f8-460b-a889-1544ed4aa00a.json


In [8]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "all-stages-max-micro-auc.tfmodel"

In [9]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()
test_set = dm.get_test_set()

In [10]:
config.proportion = {'train': 0.8, 'val':0.2, 'test': 0.0}
config.stages = [6]
config.annotation_number = 10
dm = DataManager.from_dataset(train_set, test_set, config)
dm.get_num_info()

{'test': (1178, 6583), 'train': (1057, 6116), 'val': (234, 1328)}

In [6]:
dm.get_imbalance_ratio()

,train,val,test
dorsal prothoracic pharyngeal muscle,5.017045,4.925000,5.283422
embryonic brain,1.925414,1.855422,1.915633
embryonic central nervous system,3.152941,3.086207,3.501916
embryonic dorsal epidermis,2.850909,2.703125,2.983051
embryonic head epidermis,4.693548,4.386364,4.875000
embryonic hindgut,2.394231,2.338028,2.476331
embryonic midgut,1.072407,1.043103,1.068662
embryonic ventral epidermis,3.322449,3.086207,3.519231
ubiquitous,5.303571,4.642857,5.317204
ventral nerve cord,1.824000,1.788235,1.845036


In [12]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.4
config.gamma = 2
config.use_glimpse = False
config.use_hidden_dense = False
config.read_time = 3
config.batch_size = 64

threshold = 0.4
train_data = select_label(dm.get_train_stream(), 0)
val_data = select_label(dm.get_validation_stream(), 0)
config.annotation_number = 1

In [ ]:
np.set_printoptions(formatter={'float_kind': lambda x: '%.2f' % x})
model = RNN(config, is_finetuning=False, label_weights=dm.get_imbalance_ratio().train.values[0:1])
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-4), 
                                                                           (40, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [0]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=resnet_loc, ignore=ignore_restore),
                           max_epoch=90, tower=[0])
Trainer(train_config).train()

[1104 09:12:38 @inference_runner.py:82] InferenceRunner will eval on an InputSource of size 3
[1104 09:12:38 @input_source.py:180] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1104 09:12:38 @training.py:90] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:0...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
[1104 09:12:43 @model_utils.py:47] Model Parameters: 
name                            shape                  dim
------------------------------  -----------------  -------
custom_cnn/conv1/weights:0      [3, 3, 1024, 512]  4718592
custom_cnn/conv1/biases:0       [512]                  512
custo

[1104 09:12:48 @base.py:212] Creating the session ...


In [14]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[<tf.Variable 'EMA/QueueInput/queue_size:0' shape=() dtype=float32_ref>,
 <tf.Variable 'EMA/QueueInput/queue_size/biased:0' shape=() dtype=float32_ref>,
 <tf.Variable 'EMA/QueueInput/queue_size/local_step:0' shape=() dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/weights:0' shape=(7, 7, 3, 64) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights:0' shape=(1, 1, 64, 256) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bot